# Tratamiento de datos básico con R

## Introducción

### El tratamiento de los datos en el contexto del proceso de data science
![El tratammiento de los datos es un suproceso del proceso general de la ciencia de los datos](https://www.dropbox.com/s/b3hwv654uiqr209/Obtenci%C3%B3n%20y%20Tratamiento%20de%20Datos.png?dl=1)

El tratamiento de los datos es un suproceso dentro del proceso de data science que se compone a su ves de otras actividades. En este notebook nos centramos en: 
 * La **transformación de los datos** al formato y estructura más apropiada para los algoritmos o análisis que deseamos aplicar.

### Carga de librerías

Primero cargaremos las librerías necesarias para trabajar:


In [ ]:
if (!require(pacman)){
  install.packages("pacman")
}
library(pacman)
p_load(gapminder,tidyr,dplyr,ggpubr,ggplot2)

también podrían haberse instalado cargado todas las librerías del tidyverse directamente con: 
```
install.packages("tidyverse")
library(tidyverse)
```


Seguidamente visualizamos el contenido del dataset de gapminder:


In [ ]:
gapminder

La variable gapminder es un **dataframe**: una estructura de datos rectangular  con filas y columnas similar a una tabla en una base de datos relacional, donde cada columna tiene asociado un tipo de datos concreto (entero, número en coma flotante, un factor, etc...).

La mayoría de los análisis de datos complejos en R se basan en data frames.

Más específicamente, gapminder es una variable de un subtipo concreto de dataframe usado en tidyverse, llamado **tibble** que funciona de manera más eficiente y coherente que los dataframes tradicionales de R y que es usado de manera general en todas las librerías del tidyverse. Además podemos ver el tamaño completo del tibble y el conjunto concreto de filas que se están visualizando en este momento.

En general llamaremos **observación** a cada una de las filas de un dataframe, y **variable** a cada una de las columnas.

Siempre que trabajemos con dataframes es importante tener en mente una definición clara de lo que representa una observación de ese dataframe, en este caso, dada observación representa los datos de un país para un año concreto. Así tenemos el continente al que pertenece (que se mantiene constante para cada país con los años), la esperanza de vida (columna lifeExp), y el producto interior bruto per capita (gdpPercap) que es una medida bastante común de la reiqueza del país, etc.

# Tratamiento básico de datos

## Transformando datsets salvajes en tidy data con tidyr

Aunque los principios de Tidy Data nos resulten obvios, en realidad es muy común encontrar conjuntos de datos que nos los cumples, para solucionar esto, el paquete tidyr nos porporciona 2 funciones muy importantes:

### gather

  La función  **gather()** se usa cuando los nombres de las variables codifican en realidad valores asociados a las observaciones.
  
  Un ejemplo de esto es la siguiente tabla :
  
  

In [ ]:
table4a

¿Cual es el problema de esta tabla?

Vamos a recodificar los datos de esta tabla para hacer que siga los principios de Tidy Data.

Necesitaremos transformar las variable 1999 y 2000 en otras dos variables, que llamaremos year y cases.

![resultado](https://d33wubrfki0l68.cloudfront.net/3aea19108d39606bbe49981acda07696c0c7fcd8/2de65/images/tidy-9.png)

Concretamente gather toma tres conjuntos de parámetros:

 * Las columnas originales que conforman los valores de las columnas nuevas.
 * El nombre que vamos a dar a la columna que representará el valor asociado a cada columna a transformar en sí (en nuestro caso 'year'). A este parámetro se le llama la clave, y su nombre es "key".
 * El nombre que vamos a dar a la columna que contendra los valores de las columnas a transformar (en nuestro caso 'cases'). A este parámetro se le llama el valor, y su nombre es "value".
 
 
 Los  paquete dplyr y tidyr proporciona un conjunto de pasos atómicos, que llamaremos verbos, para transformar los datos.
Cada vez que encadenemos el uso de un verbo, usaremos un pipe, que en R se escribe como **%>%**, su significado es: **toma todo lo que venga antes y aplícale la acción que viene después**, así los verbos se especifican siempre depués de un pipe.

Así nuestra operación de transformación de datos quedaría como:

In [ ]:
table4a %>% 
  gather(`1999`, `2000`, key = "year", value = "cases")

  Como se puede observar, en el resultado anterior las columnas originales han sido eliminadas.
  
  *Ejercicio*: Transforme la tabla 'table4b' de manera similar estableciendo como nombre de la columna clave 'year' y de la columna de valor 'population'. A continuación, haga un left join entre los dos resultados para obtener una tabla que combina los datos de ambas tablas con una población general y un número de casos (de una enfermedad) por país y año.
  

  
  
  ### spread
  
 La función **spread()**  es opuesta a la anterior, y se usa cuando en una columna se están representando en realidad varias variables. Por ejemplo, analicemos el contenido del dataset 'table2'.

In [ ]:
table2

Si nos fijamos detenidamente, veremos que existe más de una observación por año y país, y que en realidad es la columna 'type' la que determina el significado real de la columna count (que representa tanto el conteo de casos de la enfermedad como la población general del país).

Para que el conjunto de datos cumpla con los principios de tidy data, debemos realizar la siguiente transformación:

![Transformación a relizar](https://d33wubrfki0l68.cloudfront.net/8350f0dda414629b9d6c354f87acf5c5f722be43/bcb84/images/tidy-8.png)

Para ello usaremos la función **spread()** que toma dos parámetros:
 * La columna que contiene los nombres de las variables a crear, cuyo parámetro se llama "key", y que en nuestro caso será la columna type.
 * La columna que contiene os valores de las distintas variables, cuyo parámetro se llama "value", y que en nuestro caso dera la columna count.
 
 Por tanto, el resultado de aplicar spread a la tabla 2 sería:


In [ ]:
table2 %>%
    spread(key = type, value = count)

### Separando y juntando valores de columnas con tidyr

Otro caso muy común en conjuntos de datos que no siguen los principios de tidy data es tener columnas que contienen más de un valor. Por ejemplo observe el contenido de la tabla 3:

In [ ]:
table3

La columna rate contiene en realidad dos variables, el número de casos de la enfermedad y el número de habitantes total (del páis para ese año). 

En esto casos podemos hacer uso de la función **separate()** que toma hasta 4 parámetros:

* El nombre de la columna a separar, en nuestro caso _rate_.
* Un vector de cadenas (cuyo parámetro se llama **into**), que especifica los nombres de las nuevas columnas a crear, en nuestro caso serán _cases_ y _population_.
* (Opcionalmente) El separador a usar para delimitar los distintos valores (cuyo parámetro se llama **sep**), y que en nuestro caso será _"/"_. Formalmente, **sep** admite expresiones regulares para expresar el separador, por lo que podemos tener varios tipos de separadores distintos. Además, podemos especificar un vector de enteros y se usaran para separar el valor de la columna en base al número de caracteres que indiquemos.
* (Opcionalmente) Si queremos que se intente inferir el tipo de datos de las nuevas columnas y que se realice la converisón al generarlas podemos especificarlo mediante el parámetro **convert**, que es booleano.

Así, al aplicar la función **separate()** a la tabla 3 obendríamos:

In [ ]:
table3transformada<- table3 %>% 
  separate(rate, into = c("cases", "population"), sep = "/")
table3transformada
str(table3transformada)

**Ejercicio**: Modificar la tabla3 de manera que el resultado contenga dos columnas, cases y population de tipo numérico.

**Ejercicio**: Separar la columna year en siglo y año, de manera que por ejemplo para 1999 aparezcan 19 en siglo y 99 en año.

### Valores faltantes y la tranformación de conjuntos de datos

Una de las cosas sutiles cuando se transforman conjuntos de datos es cómo vamos a tratar los valores faltantes. 
Los valores pueden faltar en un conjunto de datos de dos maneras distintas:
* De manera explícita, es decir, aparece el valor **NA** en alguna de las celdas de nuestro conjunto de datos.
* De manera implícita, es decir, no hay ninguna celda asociada al dato faltante con un valor de **NA** pero sencillamente la fila que debería contener el valor no aparece en el conjunto de datos.

Vamos a ilustrar esto con un dataset simple:

In [ ]:
stocks <- tibble(
  year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
)
stocks

Concretamente en este dataset faltan los datos del cuarto cuatrimestre de 2015 (tiene asociado de manera explícita el valor NA en su correspondiente fila), y los datos del primer cuatrimestre de 2016 (no hay fila que contenga los valores 2016 y 1 en las columnas year y qtr respectivamente).

La manera en que está representado un dataset puele alterar la manera enque se expresan los valores faltantes, por ejemplo:

In [ ]:
stocks %>% 
  spread(year, return)

Otra herramienta interesante para hacer explícita la representación de los valores faltantes es la función **complete()**. Esta función toma una serie de nombre de columnas y calcula todas las posibles combinaciones de sus valores. Una vez hecho esto, se asegura de que el dataset contiene al menos un fila con todos esos posibles valores, y si no están, lo añade con *NA*s en el resto de columnas.
Por ejemplo:

In [ ]:
stocks %>% 
  complete(year, qtr)

Alternativamente, si queremos que no aparezcan **NA**s en el conjunto de datos transformado, podemos usar el parámetro _na.rm=TRUE_ en gather y spread, y eliminará las filas que tendría un valor faltante (*NA*).

## Tratamiento de datos usando los verbos de dyplr

Los  paquetes dplyr y tidyr proporcionan un conjunto de pasos atómicos, que llamaremos verbos, para transformar los datos.
Cada vez que encadenemos el uso de un verbo, usaremos un pipe, que en R se escribe como **%>%**, su significado es: **toma todo lo que venga antes y aplícale la acción que viene después**, así los verbos se especifican siempre depués de un pipe.

#### filter

El primer verbo de dyplr que vamos a aprender a manejar es el verbo **filter**, que se usa cuando queremos usar solo un subconjunto de nuestras observaciones que cumplen cierta condición, así para filtrar los datos de los países en 2007 tendríamos.

In [ ]:
gapminder %>% filter(year==2007)

Aquí 'year==2007' es la condición de filtrado. Cuidado, recordad que '=' en R significa asignación, mientras que '==' significa comparación.


**Ejercicio**: Os proponemos buscar los datos históricos de España (llamada "Spain" en el dataset) del año 2002 .

#### arrange

Otro verbo muy interesante es *arrange*, que permite ordenar las observaciones del dataset basandose en los valores de sus varaibles.

Por ejemplo para ordenar las observaciones de gapminder por producto interior bruto per cápita haríamos:


In [ ]:
gapminder %>% arrange(gdpPercap)

Si queremos obtener los datos en orden descendente, haríamos:

In [ ]:
gapminder %>% arrange(desc(gdpPercap))

**Ejercicio**: Ahora os inivitamos a encontrar los 10 países más ricos en términos relativos (producto interior bruto) del mundo en 2007:

#### mutate

Otro verbo especialmente interesante para el tratamiento de datos es el verbo **mutate**.
Este verbo permite cambiar los valores que existen en una de nuestras variables, basándonos en su valo y en el de las otras variables, e incluso nos permitirá crear nuevas variables calculadas (ojo el cálculo se hace una sola vez, cuando el verbo se ejecuta, y lo que se guarda en el dataframe es el valor calculado, es decir, ese valor no se auto-actualiza cuando cambiamos otras variables, no es un "trigger").


Por ejemplo, imaginemos que deseamos almacenar el tamaño de la población de los paises en lugar de en individuos en millones de individuos, podríamos hacerlo de la siguiente manera:


In [ ]:
gapminder %>% mutate(pop=pop/1000000)

**Ejercicio:** Es importante resaltar en este punto que no estamos modificando los datos almacenados en la variable gapminder, sino que se está devolviendo un nuevo dataframe con los datos cambiados. Compruébalo:

Para guardar el resultado en una nueva variable debemos asignarle un nombre:

In [ ]:
gapminder2007 <- gapminder %>% filter(year==2007)
gapminder2007

También podemos añadir una nueva variable a nuestro dataframe, sencillamente basta con darle un nombre distinto al elemento que asignamos a nuestros cálculos. Por ejemplo, podría ser interesante trabajar con el producto interior bruto total de los paises, no con su valor por habitante. Para ello:

In [ ]:
gapminder %>% mutate(gdp=gdpPercap*pop)

**Ejercicio**: Ahora podemos preguntarnos ¿Cuáles son los paises más ricos en términos absolutos (aquellos con el producto interior bruto mayor) en 2007?

**¿Por qué no está noruega ahí?**

#### sumarize

Otro verbo extremadamente útil en la fase de limpieza, transformación, y exploración de los datos es **summarize**.
Este verbo nos ayudará a entender la estructura y contenido de nuestros datos, para poder ser más efectivos a la hora de generar información valiosa o modelos a partir de los mismos.

El verbo **summarize** permite generar una única observación o dato a partir de un conjunto de ellos. Por ejemplo, nos permitirá responder a la pregunta de ¿cual era la esperanza de vida media mundial en el 2007?

In [ ]:
gapminder %>% 
  filter(year==2007) %>%  
  summarize(meanLifeExp=mean(lifeExp))

**Ejercicio**: ¿Cómo creeis que será la esperanza de vida en Asia?

**Ejercicio**:¿Y en África?

El verbo summariize no sólo nos permite calcular medias, sino que incorpora todo un conjunto de funciones de agregación de interés, por ejemplo, para calcular la población total reflejada en el dataset en 2007, podríamos hacer:

In [ ]:
gapminder %>% 
  filter(year==2007 & continent=="Africa") %>%
  summarize(meanifeExpAfrica=mean(lifeExp),
            totalPop=sum(pop))

Otras funciones de interés son _median_, _min_, _max_, _range_, etc.

#### group_by

Otro verbo muy importante para extraer y agrupar información de interés en nuestros dataframes es **group_by**, este verbo nos permite negerar varios subconjuntos de datos en base a los varlores de las variables, sobre los que depués podremos aplicar los demás verbos, y especialmente summarize.

Así si queremos sacar la experanza de vida media y pobalción total a lo largo del tiempo, podemos hacer:


In [ ]:
gapminder %>%
  group_by(year) %>%
  summarize(meanLifeExp = mean(lifeExp),
            totalPop = sum(as.numeric(pop)))

Hay un detalle sutil aquí, fíjate en que al especificar la función que calcula la población total, hemos transformado el valor a número en coma flotante (usando la función as.numeric). 

**Ejercicio**: ¿Se te ocurre porqué? Intenta hacer los mismo sin realizar la conversión y observa el resultado de la salida:

Podemos obtener la experanza de vida media y la población por continente en 2007 combinando una agrupación por continente y un flitro:

In [ ]:
gapminder %>%
  filter(year == 2007) %>%
  group_by(continent) %>%
  summarize(meanLifeExp = mean(lifeExp),
            totalPop = sum(as.numeric(pop)))

Por último, podemos agrupar por más de una columna, por ejemplo, podemos sacar la evolución de población total y esperanza de vida media a lo largo del tiempo por continente:

In [ ]:
gapminder %>%
  group_by(year, continent) %>%
  summarize(totalPop = sum(as.numeric(pop)),
            meanLifeExp = mean(lifeExp))

## Otros tratamientos de interés

A veces cuando intentamos aplicar un algorimto con un propósito concreto a un conjunto de datos nos interesa generar versiones del dataset adaptadas para el funcionamiento de dichos algoritmos que no tienen porqué seguir los principios de tidy data. En esta última sección mostraremos como realizar algunos de los casos más comunes de este tipo de transformaciones.

### Eliminar datos faltantes

Lo primero que hay que hacer es identificar el conjunto de valores que representan valores faltantes, por ejemplo muchas veces nos encontramos con valores que contienen la cadena vacía cuando en realidad el dato falta. Lo ideal es que si el dato falta, lo que apareza en el valora asociados a su observación en la variable correspondiente sea el valor **NA**. Aún cuando todos los datos que en realidad son faltantes aparecen en nuestro conjunto de datos como tales.
Algunos algoritmos no admiten la existencia de datos faltantes en el conjunto de datos que se les pasa como parámetro. Cuando se nos presenta esta situación tenemos 2 alternativas:
 
 1. *Eliminar las filas con datos faltantes*. Esto puede hacerse cuando el volumen de datos faltantes es bajo y tenemos datos suficientes como para que el algoritmo y sus resultados no se vean muy afectados por la eliminación (hay que tener en cuenta que si el patrón de datos faltantes no es aleatorio esto puede introducir un sesgo en el modelo o los resultados generados por el algoritmo a aplicar). Esta estrategia se implementa facilmente usando el verbo filter o aplicando una transformación con el parámentro _na.rm=TRUE_.
 
 **Ejercicio**: Guardar en nui nuevo dataset la versión de observaciones completas (sin NAs) del siguiente:

In [ ]:
df <- tibble(x = c(1, 2, 2),y = c(1, 2, NA))

df


 
 2. Rellenar los datos faltantes con valores "coherentes" con el resto de datos del conjunto de datos y el algoritmo que queremos aplicar. Aquí la clave está en cómo definimos esa coherencia, esencialmente hay 3 mecanismos:
* _Usar el valor de la observación inmediatamente anterior_. Esto se hace mucho cuando los datos han sido introducidos directamente por personas, que prefieren no repetir en cada observación el mismo valor (y lo dejan en blanco). Para implementar este mecanismos de tratamiento tenemos a nuestra disposición la función **fill()** que toma como parámetro un conjunto e nombres de columnas y rellena los datos faltantes en cada una con el valor que contiene la observación anterior.
* _Usar la media/mediana/moda de la columna como valor_. Esta estrategia es muy fácil de implementar usando el verbo mutate , la función replace_na, y la correspondiente función o valor a calcular. Por ejemplo:

In [ ]:
df <- tibble(x = c(1, 2, 2),y = c(1, 2, NA))

df

df %>% mutate(y = replace_na(y, mean(df$y,na.rm=T)))


 * _Usar un modelo de algún tipo (por ejemplo lineal) para general el valor en base al valor de otras columnas del conjunto de datos_. Para ello contamos con la función predict, que toma como parámetro un modelo y genera un valor, y que podemos usar junto con el verbo mutate. Por ejemplo:

In [ ]:
df <- tibble(x = c(1, 2, 2),y = c(1, 2, NA))

df

# Calucamos el modelo indicando que no tenga en cuenta los valores faltantes:
bmmodelo <- lm(y~x,data=df,na.action=na.exclude)

summary(modelo)

df %>% mutate(y=predict(modelo,.))

### Reformatear un factor como un conjunto de columnas booleanas/binarias

Algunos algoritmos no soportan el uso de factores sino que requieren el uso de columnas booleanas/binarias, podemos usar las funcion **spread**, o las funciones **mutate** junto con **ifelse** o **case_when** para realizar esta transformación.
Ejemplos:




In [ ]:
df <- tibble(x = c("a", "b", "b"),y = c(1, 2, NA))

dfspread <- df %>%  mutate(value = 1) %>%
                    spread(x, value, fill = 0)
dfspread

dfifelse <- df %>% mutate(x,a=ifelse(x=="a",1,0),b=ifelse(x=="b",1,0))
dfifelse

dfcasewhen <- df %>% mutate(
    a = case_when(
            x == "a" ~ 1,
            x == "b" ~ 0
),
    b=case_when(
            x == "b" ~ 1,
            x == "a" ~ 0
))
dfcasewhen


  
### Limpiar caracteres raros y especiales de las cadenas y los factores

Muchos algoritmos no aceptan el uso de valores con carácteres que no sean UTF-8, o incluso ASCII en las cadenas y los factores por ello es importante conoce la funciones **replace_non_ascii** (del paquete **textclean**) y **sanitize**. Por ejemplo:

In [ ]:
install.packages("textclean")
library(textclean)
astr <- "Ábcdêãçoàúü"

astr

replace_non_ascii(astr)


## Ejercicios adicionales:

### Ejercicio 1

El dataset _who_ continene información sobre casos de tuberculosis por año, país, edad del paciente, y método de diagnóstico. Los datos vienen del informe de las Organización Mundal de la Salud sobre esa enfermedad del 2014, disponbile [aquí](http://www.who.int/tb/country/data/download/en/).
Hay mucha información información en este conjunto de datos, pero es difícil trabajar con él por culpa de: 

* Columnas redundantes (country, iso2, y iso3)
* Variables codificadas de forma exótica (las columnas new_sp_m014, new_ep_m014, new_ep_f014,... tienen toda la pinta de estar codificando en realidad valores y no variables)
* Montones de valores faltantes .

En cuanto a la codificación de los valores en las variables, tras consultar con los creadores del conjunto de datos nos indicaron que:

1. Las primeras 3 letras de cada columna denotan si la columna conteine casos nuevos o antiguos (cada año). En nuestro caso solo se contaban los casos nuevo por lo que siempre será "new".

2. Las siguientes letras  denotan el tipo de tuberculosis:

* rel significa casos de recaída en la enfermedad
* ep significa casos de tuberculosis extra-pulmonar
* sn significa caoss de tuberculosis pulmonar que no pudo ser diagnosticada con un frotis pulmonar
* sp significa caoss de tuberculosis pulmonar que no pudo ser diagnosticada con un frotis pulmonar

3. La sexta letra indica el sexo de los pacientes, agrupados en machos (m) y hembras (f).

Los siguientes números codifican el grupo de edad. Se contemplan siete grupos de edad:

* 014 = 0 – 14 años
* 1524 = 15 – 24 años
* 2534 = 25 – 34 años
* 3544 = 35 – 44 años
* 4554 = 45 – 54 años
* 5564 = 55 – 64 años
* 65 = 65 o más años

In [ ]:
str(who)
who

Desgraciadamente, las personas que tomaron los datos fueron ligeramente inconsistentes, y tenemos casos en los que los valores en lugar de '_new_rel_' ponen '_newrel_'. Puede usar **mutate** y la función **stringr::str_replace(columna,"cadena_a_encontrar","reemplazo">)** para solventar este problema.

Recuerde que si desea especificar todo el conjunto de columnas entre **new_sp_m014** y **newrel_f65_** puede escribir **_new_sp_m014:newrel_f65**.

1. Transforme el conjunto de datos _who_ en un conjunto llamado _tidyWho_ que siga los principios de tidy data.

2. Compruebe si realmente las columnas country, iso2 y iso3 son redundates.

3. Comprueba el número de casos por país, año, y sexo. Piense como podríamos representar visualmente dicha información y describa dicha gráfica (si lo desea puede intentar generarla usando la librerías básicas de R o ggplot2).